# Submit a single training run to AML Compute

In [16]:
from azureml.core import Workspace, Experiment
from azureml.train.dnn import TensorFlow
from azureml.widgets import RunDetails

### Initialize Azure Machine Learning Workspace

In [8]:
ws = Workspace.from_config()

gpu_cluster = ws.compute_targets['gpu-cluster']
food_data = ws.datastores['food_images']

In [84]:
script_arguments = {
    "--data-dir": food_data.as_mount(),
    "--epochs": 50,
    "--learning-rate": 1e-4,
    "--minibatch-size": 64,
    "--optimizer": 'adadelta'
}

tf_config = TensorFlow(source_directory="..",
                       entry_script='code/train/train.py',
                       script_params=script_arguments,
                       compute_target=gpu_cluster,
                       conda_packages=['pillow', 'pandas'],
                       pip_packages=['click', 'seaborn'],
                       use_docker=True,
                       use_gpu=True
                      )

# Run on subset of food categories
tf_config.run_config.arguments.extend(['apple_pie', 
                                       'baby_back_ribs', 
                                       'baklava', 
                                       'beef_carpaccio'])

WARNING - framework_version is not specified, defaulting to version 1.13.


In [85]:
single_exp = Experiment(ws, "single_run")
tf_single_run = single_exp.submit(tf_config)

In [86]:
RunDetails(tf_single_run).show()

A Jupyter Widget